## Section

### Import library

In [ ]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout, Conv2D, AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Flatten

from tensorflow.keras.optimizers import Adam

### Load preprocessed data

In [ ]:
preprocessed_data_dir = '/content/drive/MyDrive/Informatika/Semester 7/Pembelajaran Mesin/Tugas Kelompok Praktikum/Hasil Preprocessing - Skenario Praktikum Modul 2'

x_train = np.load(preprocessed_data_dir + '/x_train.npy')
x_val = np.load(preprocessed_data_dir + '/x_val.npy')
x_test = np.load(preprocessed_data_dir + '/x_test.npy')

y_train = np.load(preprocessed_data_dir + '/y_train.npy')
y_val = np.load(preprocessed_data_dir + '/y_val.npy')
y_test = np.load(preprocessed_data_dir + '/y_test.npy')

In [ ]:
print("Train Data =", x_train.shape)
print("Train Label =", y_train.shape)
print('-'*30)
print("Test Data =", x_test.shape)
print("Test Label =", y_test.shape)
print('-'*30)
print("Val Data =", x_val.shape)
print("Val Label =", y_val.shape)

Train Data = (5216, 250, 250, 3)
Train Label = (5216,)
------------------------------
Test Data = (624, 250, 250, 3)
Test Label = (624,)
------------------------------
Val Data = (16, 250, 250, 3)
Val Label = (16,)


### Modelling

#### Skema 1

In [ ]:
# Feature Extraction Layer
model = Sequential()

model.add(InputLayer(input_shape=[250,250,3]))
model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(GlobalMaxPooling2D())
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

# Print model summary
print(model.summary())

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 250, 250, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 125, 125, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
global_max_pooling2d (Global (None, 64)                0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

In [ ]:
H = model.fit(x_train, y_train,
              batch_size=20,
              epochs=20,
              validation_data=(x_test, y_test)
              )

Epoch 1/20
261/261 [==============================] - 304s 1s/step - loss: 0.4115 - acc: 0.8198 - val_loss: 0.9248 - val_acc: 0.7308
Epoch 2/20
261/261 [==============================] - 294s 1s/step - loss: 0.2275 - acc: 0.9153 - val_loss: 0.9650 - val_acc: 0.7099
Epoch 3/20
261/261 [==============================] - 297s 1s/step - loss: 0.1934 - acc: 0.9271 - val_loss: 0.6737 - val_acc: 0.7324
Epoch 4/20
261/261 [==============================] - 293s 1s/step - loss: 0.1789 - acc: 0.9314 - val_loss: 1.0396 - val_acc: 0.7003
Epoch 5/20
261/261 [==============================] - 293s 1s/step - loss: 0.1574 - acc: 0.9344 - val_loss: 0.7035 - val_acc: 0.7548
Epoch 6/20
261/261 [==============================] - 291s 1s/step - loss: 0.1512 - acc: 0.9419 - val_loss: 0.6768 - val_acc: 0.7692
Epoch 7/20
261/261 [==============================] - 291s 1s/step - loss: 0.1345 - acc: 0.9459 - val_loss: 1.3365 - val_acc: 0.7019
Epoch 8/20
261/261 [==============================] - 293s 1s/step - 

In [ ]:
import json

target_dir = '/content/drive/MyDrive/Informatika/Semester 7/Pembelajaran Mesin/Tugas Kelompok Praktikum/Modelling - Skenario Praktikum Modul 2/Hasil Modelling - Skenario Praktikum Modul 2'

# simpan history model
history_dict = H.history
json.dump(history_dict, open(target_dir + '/epoch_20_history_model_skema_1.json', 'w'))

# simpan model dan weight-nya
model.save(target_dir + '/epoch_20_model_skema_1.h5')
model.save_weights(target_dir + '/epoch_20_model_skema_1_weights.h5')

In [ ]:
from sklearn.metrics import classification_report

pred = model.predict(x_val)
labels = (pred > 0.5).astype(np.int)

print('Model Skema 1')
print(classification_report(y_val, labels))

Model Skema 1
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         8
           1       0.67      1.00      0.80         8

    accuracy                           0.75        16
   macro avg       0.83      0.75      0.73        16
weighted avg       0.83      0.75      0.73        16



#### Skema 2

In [ ]:
# Feature Extraction Layer
model = Sequential()

model.add(InputLayer(input_shape=[250,250,3]))
model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(AveragePooling2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(AveragePooling2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(GlobalAveragePooling2D())
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

# Print model summary
print(model.summary())

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['acc'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 250, 250, 16)      448       
_________________________________________________________________
average_pooling2d (AveragePo (None, 125, 125, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
average_pooling2d_1 (Average (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
global_average_pooling2d (Gl (None, 64)                0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)               

In [32]:
H = model.fit(x_train, y_train,
              batch_size=20,
              epochs=20,
              validation_data=(x_test, y_test)
              )

Epoch 1/20
261/261 [==============================] - 273s 1s/step - loss: 0.5753 - acc: 0.7410 - val_loss: 0.6296 - val_acc: 0.6250
Epoch 2/20
261/261 [==============================] - 265s 1s/step - loss: 0.4865 - acc: 0.7425 - val_loss: 0.7348 - val_acc: 0.6250
Epoch 3/20
261/261 [==============================] - 265s 1s/step - loss: 0.4662 - acc: 0.7421 - val_loss: 0.7129 - val_acc: 0.6250
Epoch 4/20
261/261 [==============================] - 266s 1s/step - loss: 0.4232 - acc: 0.7694 - val_loss: 0.6785 - val_acc: 0.6859
Epoch 5/20
261/261 [==============================] - 265s 1s/step - loss: 0.3811 - acc: 0.8158 - val_loss: 0.8184 - val_acc: 0.6747
Epoch 6/20
261/261 [==============================] - 266s 1s/step - loss: 0.3551 - acc: 0.8414 - val_loss: 0.8609 - val_acc: 0.6907
Epoch 7/20
261/261 [==============================] - 265s 1s/step - loss: 0.3451 - acc: 0.8476 - val_loss: 0.9462 - val_acc: 0.6699
Epoch 8/20
261/261 [==============================] - 265s 1s/step - 

In [34]:
import json

target_dir = '/content/drive/MyDrive/Informatika/Semester 7/Pembelajaran Mesin/Tugas Kelompok Praktikum/Modelling - Skenario Praktikum Modul 2/Hasil Modelling - Skenario Praktikum Modul 2'

# simpan history model
history_dict = H.history
json.dump(history_dict, open(target_dir + '/epoch_20_history_model_skema_2.json', 'w'))

# simpan model dan weight-nya
model.save(target_dir + '/epoch_20_model_skema_2.h5')
model.save_weights(target_dir + '/epoch_20_model_skema_2_weights.h5')

In [33]:
from sklearn.metrics import classification_report

pred = model.predict(x_val)
labels = (pred > 0.5).astype(np.int)

print('Model Skema 2')
print(classification_report(y_val, labels))

Model Skema 2
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         8
           1       0.67      1.00      0.80         8

    accuracy                           0.75        16
   macro avg       0.83      0.75      0.73        16
weighted avg       0.83      0.75      0.73        16

